In [ ]:
"""
Purpose: To create the function that does the soma touching 
skeletonization correctly and doesn't double skeletonize

"""

In [ ]:
from os import sys
sys.path.append("../meshAfterParty/")

In [ ]:
import skeleton_utils as sk
import soma_extraction_utils as soma_utils
import trimesh

from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np
from pathlib import Path

import time
import os
import pathlib

from tqdm.notebook import tqdm

In [ ]:
current_neuron = trimesh.load_mesh("../test_neurons/multi_soma_example.off")
segment_id = 12345
#current_neuron.show()

# Do the soma detection

In [ ]:
soma_mesh_list,run_time,total_soma_list_sdf = soma_utils.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)

total_soma = sk.combine_meshes(soma_mesh_list)
total_soma.show()

# do the skeletonization

In [ ]:
from skeleton_utils import *

main_mesh_total = current_neuron
segment_id = 12345
soma_mesh_list
mesh_base_path=""
current_name=""
filter_end_node_length=5000
sig_th_initial_split=15


    
if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id} so just one mesh")
    soma_mesh_list_centers = []
else:
    #compute the soma centers
    print(f"Soma List = {soma_mesh_list}")

    soma_mesh_list_centers = find_soma_centroids(soma_mesh_list)
    print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")


split_meshes = split_significant_pieces(
                        main_mesh_total,
                        significance_threshold=sig_th_initial_split,
                        print_flag=False)


    
#returns the index of the split_meshes index that contains each soma    
containing_mesh_indices = find_soma_centroid_containing_meshes(soma_mesh_list_centers,
                                        split_meshes)

non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                 if i not in list(containing_mesh_indices.values())]


# ***** this part will have a repeat of the meshes that contain the soma *** #
soma_touching_meshes = dict([(i,split_meshes[m_i]) 
                             for i,m_i in containing_mesh_indices.items()])


In [ ]:
containing_mesh_indices

In [ ]:
    
#Adding the step that will filter away any pieces that are inside the soma
if len(non_soma_touching_meshes) > 0 and len(soma_mesh_list) > 0:
    non_soma_touching_meshes = soma_utils.filter_away_inside_soma_pieces(soma_mesh_list,non_soma_touching_meshes)


print(f"# of non soma touching seperate meshes = {len(non_soma_touching_meshes)}")
print(f"# of soma containing seperate meshes = {len(list(soma_touching_meshes.keys()))}")


# setting the base path and the current name
if mesh_base_path == "":
    mesh_base_path = Path(f"./{segment_id}")
else:
    mesh_base_path = Path(mesh_base_path)

if current_name == "":
    current_name = f"{segment_id}"

if mesh_base_path.exists():
    rmtree(str(mesh_base_path.absolute()))
mesh_base_path.mkdir(parents=True,exist_ok=True)
print(list(mesh_base_path.iterdir()))


In [ ]:
def grouping_containing_mesh_indices(containing_mesh_indices):
    """
    Purpose: To take a dictionary that maps the soma indiece to the 
             mesh piece containing the indices: {0: 0, 1: 0}
             
             and to rearrange that to a dictionary that maps the mesh piece
             to a list of all the somas contained inside of it 
             
    Pseudocode: 
    1) get all the unique mesh pieces and create a dictionary with an empty list
    2) iterate through the containing_mesh_indices dictionary and add each
       soma index to the list of the containing mesh index
    3) check that none of the lists are empty or else something has failed
             
    """
    
    unique_meshes = np.unique(list(containing_mesh_indices.values()))
    mesh_groupings = dict([(i,[]) for i in unique_meshes])
    
    #2) iterate through the containing_mesh_indices dictionary and add each
    #   soma index to the list of the containing mesh index
    
    for soma_idx, mesh_idx in containing_mesh_indices.items():
        mesh_groupings[mesh_idx].append(soma_idx)
    
    #3) check that none of the lists are empty or else something has failed
    len_lists = [len(k) for k in mesh_groupings.values()]
    
    if 0 in len_lists:
        raise Exception("One of the lists is empty when grouping somas lists")
        
    return mesh_groupings
    

In [ ]:
def recursive_soma_skeletonization(main_mesh,
                                  soma_mesh_list,
                                soma_mesh_list_indexes,
                                   mesh_base_path="./temp_folder",
                                  soma_mesh_list_centers=[],
                                  ):
    """
    Parameters:
    Mesh piece 
    The soma centers list and meshes list contained somewhere within the mesh piece
    
        Algorithm
    1) Start with the first soma and subtract from mesh
    2) Find all of the disconnected mesh pieces
    3) If there is still a soma piece that has not been processed, 
    find mesh pieces and all the somas that are contained within that
    4) Send Each one of those mesh pieces and soma lists
    to the recursive_soma_skeletonization (these will return skeletons)
    5) For all other pieces that do not have a soma do skeleton of branch
    6) Do soma skeleton stitching using all the branches and those returning
    from step 4
    7) return skeleton

    """
    print("\n\n Inside New skeletonization recursive calls\n\n")
    
    if len(soma_mesh_list) == 0:
        raise Exception("soma_mesh_list was empty")
    else:
        soma_mesh_list = list(soma_mesh_list)
    
    #0) If don't have the soma_mesh centers then calculate
    if len(soma_mesh_list_centers) != len(soma_mesh_list):
        soma_mesh_list_centers = find_soma_centroids(soma_mesh_list)
    
    #1) Start with the first soma and subtract from mesh
    #2) Find all of the disconnected mesh pieces
    current_soma = soma_mesh_list.pop(0)
    current_soma_index = soma_mesh_list_indexes.pop(0)
    current_soma_center = soma_mesh_list_centers.pop(0)
    mesh_pieces = subtract_soma(current_soma,main_mesh)
    print(f"currently working on soma index {current_soma_index}")
    
    print(f"mesh_pieces after the soma subtraction = {len(mesh_pieces)}")
    
    #3) If there is still a soma piece that has not been processed, 
    total_soma_skeletons = []
    
    if len(soma_mesh_list) > 0:
        #find mesh pieces and all the somas that are contained within that
        containing_mesh_indices = find_soma_centroid_containing_meshes(
                                            soma_mesh_list_centers,
                                            mesh_pieces
        )
        
        # rearrange into lists of somas per mesh soma 
        meshes_mapped_to_somas = grouping_containing_mesh_indices(containing_mesh_indices)
        
        #get all of the other mesh pieces that weren't a part of the soma containing
        mesh_pieces_with_soma = list(meshes_mapped_to_somas.keys())
        non_soma_branches = [k for i,k in enumerate(mesh_pieces) if i not in mesh_pieces_with_soma]

        print(f"meshes_mapped_to_somas = {meshes_mapped_to_somas}")
        
        #recursive call to the function to find all those skeletons for the 
        #mesh groupings 
        for mesh_idx,soma_list in meshes_mapped_to_somas.items():
            mesh_soma_list = [k for i,k in enumerate(soma_mesh_list) if i in soma_list]
            mesh_soma_list_indexes = [k for i,k in enumerate(soma_mesh_list_indexes) if i in soma_list]
            mesh_soma_list_centers = [k for i,k in enumerate(soma_mesh_list_centers) if i in soma_list]
            
            print(f"mesh_soma_list = {mesh_soma_list}\n"
                f"mesh_soma_list_indexes = {mesh_soma_list_indexes}\n"
                 f"mesh_soma_list_centers = {mesh_soma_list_centers}\n")
            
            soma_mesh_skeleton = recursive_soma_skeletonization(
                                  mesh_pieces[mesh_idx],
                                  soma_mesh_list=mesh_soma_list,
                                    soma_mesh_list_indexes = mesh_soma_list_indexes,
                                  soma_mesh_list_centers=mesh_soma_list_centers,
                                mesh_base_path=mesh_base_path
            )
            
            total_soma_skeletons.append(soma_mesh_skeleton)
        
        
        
    
    else:
        non_soma_branches = mesh_pieces
    
    
    print(f"non_soma_branches = {len(non_soma_branches)}")
    print(f"mesh_pieces = {len(mesh_pieces)}")

    
    #5) For all other pieces that do not have a soma do skeleton of branch
    for dendrite_index,picked_dendrite in enumerate(non_soma_branches):
        dendrite_name=current_name + f"_soma_{current_soma_index}_branch_{dendrite_index}"
        
        print(f"\n\nWorking on {dendrite_name}")
        stitched_dendrite_skeleton = skeletonize_connected_branch(picked_dendrite,
                                                       output_folder=mesh_base_path,
                                                       name=dendrite_name,
                                                        skeleton_print = True)
        
        if len(stitched_dendrite_skeleton)<=0:
                print(f"*** Dendrite {dendrite_index} did not have skeleton computed***")
        else: 
            total_soma_skeletons.append(stitched_dendrite_skeleton)
    
    #stitching together the soma parts:
    soma_stitched_skeleton = soma_skeleton_stitching(total_soma_skeletons,current_soma)
    
    #return the stitched skeleton
    return soma_stitched_skeleton
    
    

In [ ]:
try:
    soma_mesh_list_centers_cp
    soma_mesh_list_cp
    soma_mesh_list_indexes_cp
except:
    soma_mesh_list_centers_cp = soma_mesh_list_centers.copy()
    soma_mesh_list_cp = soma_mesh_list.copy()
    soma_mesh_list_indexes_cp = list(np.arange(len(soma_mesh_list)))

# to reload them 
soma_mesh_list_centers = soma_mesh_list_centers_cp
soma_mesh_list = soma_mesh_list_cp
soma_mesh_list_indexes = soma_mesh_list_indexes_cp



In [ ]:
containing_mesh_indices

In [ ]:
soma_utils = reload(soma_utils)

sk = reload(sk)
from skeleton_utils import *

# do the recursive skeletonizatio
# rearrange into lists of somas per mesh soma 
meshes_mapped_to_somas = grouping_containing_mesh_indices(containing_mesh_indices)

print(f"meshes_mapped_to_somas = {meshes_mapped_to_somas}")

final_soma_skeletons_list = []
#recursive call to the function to find all those skeletons for the 
#mesh groupings 
for mesh_idx,soma_list in meshes_mapped_to_somas.items():
    mesh_soma_list = [k for i,k in enumerate(soma_mesh_list) if i in soma_list]
    mesh_soma_list_indexes = [k for i,k in enumerate(soma_mesh_list_indexes) if i in soma_list]
    mesh_soma_list_centers = [k for i,k in enumerate(soma_mesh_list_centers) if i in soma_list]

    print(f"mesh_soma_list = {mesh_soma_list}\n"
        f"mesh_soma_list_indexes = {mesh_soma_list_indexes}\n"
         f"mesh_soma_list_centers = {mesh_soma_list_centers}\n")


    soma_mesh_skeleton = recursive_soma_skeletonization(
                                  split_meshes[mesh_idx],
                                  soma_mesh_list=mesh_soma_list,
                                    soma_mesh_list_indexes = mesh_soma_list_indexes,
                                  soma_mesh_list_centers=mesh_soma_list_centers,
                                mesh_base_path=mesh_base_path
    )
    
    final_soma_skeletons_list.append(soma_mesh_skeleton)

final_soma_skeletons_list

In [ ]:
sk.graph_skeleton_and_mesh(current_neuron.vertices,
                          current_neuron.faces,
                          edge_coordinates=final_soma_skeletons_list[0],
                          html_path="debuging_multi_skeleton.html")

# Trying out the new skeletonizatoin function that can handle multiple soma

In [5]:
from os import sys
sys.path.append("../meshAfterParty/")

In [2]:
import skeleton_utils as sk
import soma_extraction_utils as soma_utils
import trimesh

from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np
from pathlib import Path

import time
import os
import pathlib

from tqdm.notebook import tqdm

In [3]:
current_neuron = trimesh.load_mesh("../test_neurons/multi_soma_example.off")
segment_id = 12345
#current_neuron.show()

# --- Do the soma detection

In [6]:
soma_mesh_list,run_time,total_soma_list_sdf = soma_utils.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 535
xvfb-run -n 535 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/12345/neuron_12345.off -o /notebooks/Platinum_Skeletonization_vp3/12345/neuron_12345_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/12345/decimation_meshlab_25549649.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(139795, 3), faces.shape=(284334, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(139795, 3), faces.shape=(284334, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Skeletonization_vp3/12345/neuron_12345_decimated_largest_piece.off
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Us

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(151132, 3), faces.shape=(302272, 3))>, <trimesh.Trimesh(vertices.shape=(42551, 3), faces.shape=(85098, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(151132, 3), faces.shape=(302272, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 5271
xvfb-run -n 5271 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/12345/decimation_meshlab_25167091.mls
done exporting decimated mesh: neuron_12345_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh fix: 0.0002856254577636

face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_549622.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_549622_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_105103.mls is being deleted....
Inside sphere validater: ratio_val = 2.882922788675577
----- working on mesh after poisson #1: <trimesh.Trimesh(vertices.shape=(42551, 3), faces.shape=(85098, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 5990
xvfb-run -n 5990 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/12345/neuron_12345_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/12345/decimation_meshlab_25167091.mls
done exporting decimated mesh: neuron_12345_decimated_largest_piece_poisson_lar

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_613536.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_613536_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_870503.mls is being deleted....
Inside sphere validater: ratio_val = 3.0502651518210446



 Total time for run = 118.0660388469696


In [7]:
total_soma = sk.combine_meshes(soma_mesh_list)
total_soma.show()

# -- Do the skeletonization

In [13]:
from importlib import reload
sk = reload(sk)

In [14]:
multi_soma_skeleton = sk.skeletonize_neuron(main_mesh_total=current_neuron,
                        segment_id = 12345,
                        soma_mesh_list = soma_mesh_list,
                       mesh_base_path="",
                       current_name="",
                       filter_end_node_length=5000,
                       sig_th_initial_split=15,

                        )

Not computing soma because list already given: [<trimesh.Trimesh(vertices.shape=(3177, 3), faces.shape=(6235, 3))>, <trimesh.Trimesh(vertices.shape=(2138, 3), faces.shape=(4236, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(3177, 3), faces.shape=(6235, 3))>, <trimesh.Trimesh(vertices.shape=(2138, 3), faces.shape=(4236, 3))>]
soma_mesh_list_centers = [array([756915.02275732, 958987.26197671, 874182.12348127]), array([864025.20827876, 994593.71529467, 859608.41014967])]
There were 0 pieces found after size threshold
THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold
# of non soma touching seperate meshes = 0
# of soma containing seperate meshes = 1
contents of soma containing seperate meshes = [0]
[]


 ---- Working on soma touching skeletons ------
meshes_mapped_to_somas = {0: [0, 1]}
mesh_soma_list = [<trimesh.Trimesh(vertices.shape=(3177, 3), faces.shape=(6235, 3))>, <trimesh.Trimesh(vertices.shape=(2138, 3), faces.shape=(4236, 3))>]
mesh_soma_list_indexes = [0, 1

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Signifiant mesh pieces of 50 size after poisson = 323
     Starting Calcification
-----Time for Running Calcification = 6.414720296859741
Before mesh subtraction number of skeleton edges = 2798
Inside mesh subtraction, len(main_mesh_bbox_restricted.faces) = 176175


No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?



Total Mesh subtraction time = 61.2784


...


Total Mesh subtraction time = 0.1591



After cgal process the un-stitched skeleton has shape (37, 2, 3)
len_subgraphs AT BEGINNING of the loop


all graph is one component!
Total time for skeleton stitching = 0.02994823455810547
soma_connecting_skeleton[0].shape = (1, 2, 3)
total_soma_skeletons[0].shape = (11293, 2, 3)
Total time for soma touching skeletons: 661.2366328239441


 ---- Working on non-soma touching skeletons ------
Time for non-soma skeletons = 2.1457672119140625e-06
No skeletons to stack so returning empty list
only one skeleton so no stacking needed
only one skeleton so no stacking needed

len_subgraphs AT BEGINNING of the loop


all graph is one component!

Total time for skeleton stitching = 1.1140763759613037
Using Distance measure skeletal_distance


Total time for skeleton clean 256.24459433555603




Total time for whole skeletonization of neuron = 919.5366291999817


In [19]:
sk.graph_skeleton_and_mesh(
    current_neuron.vertices,
    current_neuron.faces,
    edge_coordinates=multi_soma_skeleton,
    html_path="multi_soma_fixed.html"
)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [16]:
multi_soma_skeleton.shape

(17413, 2, 3)

# >3 Soma Test

In [36]:
current_neuron_path = "../test_neurons/91208269699305759_6_soma.off"
current_neuron = trimesh.load_mesh(current_neuron_path)
segment_id = 91208269699305759

In [37]:
soma_mesh_list,run_time,total_soma_list_sdf = soma_utils.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)

total_soma = sk.combine_meshes(soma_mesh_list)
total_soma.show()

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8680
xvfb-run -n 8680 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759.off -o /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/91208269699305759/decimation_meshlab_25818417.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(381961, 3), faces.shape=(769132, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(381961, 3), faces.shape=(769132, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_dec

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(37385, 3), faces.shape=(74766, 3))>, <trimesh.Trimesh(vertices.shape=(23216, 3), faces.shape=(46432, 3))>, <trimesh.Trimesh(vertices.shape=(19723, 3), faces.shape=(39442, 3))>, <trimesh.Trimesh(vertices.shape=(16677, 3), faces.shape=(33350, 3))>, <trimesh.Trimesh(vertices.shape=(14710, 3), faces.shape=(29416, 3))>, <trimesh.Trimesh(vertices.shape=(5309, 3), faces.shape=(10614, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(37385, 3), faces.shape=(74766, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8203
xvfb-run -n 8203 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner_decimated.off -s /

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_884049.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_884049_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_325595.mls is being deleted....
Inside sphere validater: ratio_val = 2.9533266408769037
----- working on mesh after poisson #1: <trimesh.Trimesh(vertices.shape=(23216, 3), faces.shape=(46432, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 4004
xvfb-run -n 4004 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/91208269699305759/decimation_meshlab_25795546.mls
done exporting dec

...

removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_581024.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_581024_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_705591.mls is being deleted....
Inside sphere validater: ratio_val = 2.1875867210106636
----- working on mesh after poisson #4: <trimesh.Trimesh(vertices.shape=(14710, 3), faces.shape=(29416, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 6873
xvfb-run -n 6873 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/91208269699305759/decimation_meshlab_25795546.mls
done exporting dec

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_417664.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_417664_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_77221.mls is being deleted....
Inside sphere validater: ratio_val = 2.665791669992408
----- working on mesh after poisson #5: <trimesh.Trimesh(vertices.shape=(5309, 3), faces.shape=(10614, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 3605
xvfb-run -n 3605 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/91208269699305759/neuron_91208269699305759_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/91208269699305759/decimation_meshlab_25795546.mls
done exporting decima

face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_851890.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_851890_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_132854.mls is being deleted....
Inside sphere validater: ratio_val = 2.3788631493039136



 Total time for run = 238.9245731830597


In [38]:
multi_soma_skeleton = sk.skeletonize_neuron(main_mesh_total=current_neuron,
                        segment_id = segment_id,
                        soma_mesh_list = soma_mesh_list,
                       mesh_base_path="",
                       current_name="",
                       filter_end_node_length=5000,
                       sig_th_initial_split=15,

                        )

Not computing soma because list already given: [<trimesh.Trimesh(vertices.shape=(1867, 3), faces.shape=(3663, 3))>, <trimesh.Trimesh(vertices.shape=(971, 3), faces.shape=(1904, 3))>, <trimesh.Trimesh(vertices.shape=(1051, 3), faces.shape=(2059, 3))>, <trimesh.Trimesh(vertices.shape=(920, 3), faces.shape=(1772, 3))>, <trimesh.Trimesh(vertices.shape=(1028, 3), faces.shape=(2008, 3))>, <trimesh.Trimesh(vertices.shape=(856, 3), faces.shape=(1699, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(1867, 3), faces.shape=(3663, 3))>, <trimesh.Trimesh(vertices.shape=(971, 3), faces.shape=(1904, 3))>, <trimesh.Trimesh(vertices.shape=(1051, 3), faces.shape=(2059, 3))>, <trimesh.Trimesh(vertices.shape=(920, 3), faces.shape=(1772, 3))>, <trimesh.Trimesh(vertices.shape=(1028, 3), faces.shape=(2008, 3))>, <trimesh.Trimesh(vertices.shape=(856, 3), faces.shape=(1699, 3))>]
soma_mesh_list_centers = [array([ 899040.220782  , 1036332.62613819,  876844.76293519]), array([811080.13295572, 662131.63882595,

mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(1083665, 3), faces.shape=(2169362, 3))>, <trimesh.Trimesh(vertices.shape=(40011, 3), faces.shape=(80059, 3))>, <trimesh.Trimesh(vertices.shape=(21691, 3), faces.shape=(43406, 3))>, <trimesh.Trimesh(vertices.shape=(11608, 3), faces.shape=(23255, 3))>, <trimesh.Trimesh(vertices.shape=(1325, 3), faces.shape=(2623, 3))>]
There were 5 pieces found after size threshold
Mesh piece 0 OUTSIDE all meshes (corrected)
Mesh piece 1 OUTSIDE all meshes (corrected)
Mesh piece 2 OUTSIDE all meshes (corrected)
Mesh piece 3 OUTSIDE all meshes (corrected)
Mesh piece 4 OUTSIDE all meshes (corrected)
mesh pieces in subtact soma AFTER the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(1083665, 3), faces.shape=(2169362, 3))>, <trimesh.Trimesh(vertices.shape=(40011, 3), faces.shape=(80059, 3))>, <trimesh.Trimesh(vertices.shape=(21691, 3), faces.shape=(43406, 3))>, <trimesh.Trimesh(vertices.shape=(116

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Signifiant mesh pieces of 50 size after poisson = 1749
     Starting Calcification
-----Time for Running Calcification = 11.401148319244385
Before mesh subtraction number of skeleton edges = 16777
Inside mesh subtraction, len(main_mesh_bbox_restricted.faces) = 1321313


No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?
No adjacent faces detected! Did you merge vertices?


...


Total Mesh subtraction time = 0.1767



After cgal process the un-stitched skeleton has shape (47, 2, 3)
len_subgraphs AT BEGINNING of the loop


all graph is one component!
Total time for skeleton stitching = 0.030750036239624023
soma_connecting_skeleton[0].shape = (1, 2, 3)
total_soma_skeletons[0].shape = (70018, 2, 3)
Total time for soma touching skeletons: 3417.243022918701


 ---- Working on non-soma touching skeletons ------
Time for non-soma skeletons = 2.6226043701171875e-06
No skeletons to stack so returning empty list
only one skeleton so no stacking needed
only one skeleton so no stacking needed

len_subgraphs AT BEGINNING of the loop


all graph is one component!

Total time for skeleton stitching = 3.407339096069336
Using Distance measure skeletal_distance


Total time for skeleton clean 3064.1277091503143




Total time for whole skeletonization of neuron = 6489.796130895615


In [39]:
sk.graph_skeleton_and_mesh(
    current_neuron.vertices,
    current_neuron.faces,
    edge_coordinates=multi_soma_skeleton,
    html_path=f"{segment_id}_5_somas.html"
    
)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [41]:
sk.save_skeleton_cgal(multi_soma_skeleton,f"{segment_id}_5_skeleton.cgal")

# >3 Soma Test

In [47]:
segment_id = 74115880476384817
current_neuron_path = "../test_neurons/" + f"{segment_id}_lots_of_merge_chunks.off"

current_neuron = trimesh.load_mesh(current_neuron_path)
export_name = f"{segment_id}_lots_of_merge_chunks"

In [45]:
soma_mesh_list,run_time,total_soma_list_sdf = soma_utils.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)

total_soma = sk.combine_meshes(soma_mesh_list)
total_soma.show()

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8301
xvfb-run -n 8301 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/74115880476384817/neuron_74115880476384817.off -o /notebooks/Platinum_Skeletonization_vp3/74115880476384817/neuron_74115880476384817_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/74115880476384817/decimation_meshlab_25140638.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(118728, 3), faces.shape=(250796, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(118728, 3), faces.shape=(250796, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Skeletonization_vp3/74115880476384817/neuron_74115880476384817_dec

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(148293, 3), faces.shape=(296912, 3))>, <trimesh.Trimesh(vertices.shape=(31818, 3), faces.shape=(63816, 3))>, <trimesh.Trimesh(vertices.shape=(16458, 3), faces.shape=(32948, 3))>, <trimesh.Trimesh(vertices.shape=(7845, 3), faces.shape=(15686, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(148293, 3), faces.shape=(296912, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 5104
xvfb-run -n 5104 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/74115880476384817/neuron_74115880476384817_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/74115880476384817/neuron_74115880476384817_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/74115880476384817/decimation_meshlab_25160052.mls
done exporting decimated mesh: neuron_741158804

face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_951393.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_951393_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_155204.mls is being deleted....
Inside sphere validater: ratio_val = 3.0009997733757485
Using Poisson Surface Reconstruction to make mesh watertight
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8455
xvfb-run -n 8455 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_961949.off -o /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_961949_poisson.off -s /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_728811.mls
removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_961949.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_961949_poisson.off

face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_491296.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_491296_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_305432.mls is being deleted....
Inside sphere validater: ratio_val = 2.300713553609563
----- working on mesh after poisson #2: <trimesh.Trimesh(vertices.shape=(16458, 3), faces.shape=(32948, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 6856
xvfb-run -n 6856 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/74115880476384817/neuron_74115880476384817_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/74115880476384817/neuron_74115880476384817_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/74115880476384817/decimation_meshlab_25160052.mls
done exporting deci

In [50]:
sk = reload(sk)

In [51]:
multi_soma_skeleton = sk.skeletonize_neuron(main_mesh_total=current_neuron,
                        segment_id = segment_id,
                        soma_mesh_list = soma_mesh_list,
                       mesh_base_path="",
                       current_name="",
                       filter_end_node_length=5000,
                       sig_th_initial_split=15,

                        )

Not computing soma because list already given: [<trimesh.Trimesh(vertices.shape=(1770, 3), faces.shape=(3418, 3))>, <trimesh.Trimesh(vertices.shape=(1886, 3), faces.shape=(3743, 3))>, <trimesh.Trimesh(vertices.shape=(3702, 3), faces.shape=(7314, 3))>, <trimesh.Trimesh(vertices.shape=(2194, 3), faces.shape=(4191, 3))>, <trimesh.Trimesh(vertices.shape=(924, 3), faces.shape=(1757, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(1770, 3), faces.shape=(3418, 3))>, <trimesh.Trimesh(vertices.shape=(1886, 3), faces.shape=(3743, 3))>, <trimesh.Trimesh(vertices.shape=(3702, 3), faces.shape=(7314, 3))>, <trimesh.Trimesh(vertices.shape=(2194, 3), faces.shape=(4191, 3))>, <trimesh.Trimesh(vertices.shape=(924, 3), faces.shape=(1757, 3))>]
soma_mesh_list_centers = [array([332426.68446328, 792776.79734463, 802583.87429378]), array([344673.09013786, 809894.89369035, 808824.6630965 ]), array([332228.63184765, 735740.22595894, 799679.70831983]), array([314052.09630811, 874881.24380127, 803389.3686873

mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(253393, 3), faces.shape=(514598, 3))>, <trimesh.Trimesh(vertices.shape=(29808, 3), faces.shape=(59719, 3))>, <trimesh.Trimesh(vertices.shape=(18501, 3), faces.shape=(37028, 3))>, <trimesh.Trimesh(vertices.shape=(1453, 3), faces.shape=(2773, 3))>, <trimesh.Trimesh(vertices.shape=(1180, 3), faces.shape=(2671, 3))>, <trimesh.Trimesh(vertices.shape=(725, 3), faces.shape=(1414, 3))>, <trimesh.Trimesh(vertices.shape=(320, 3), faces.shape=(629, 3))>]
There were 7 pieces found after size threshold
Mesh piece 0 OUTSIDE all meshes (corrected)
Mesh piece 1 OUTSIDE all meshes (corrected)
Mesh piece 2 OUTSIDE all meshes (corrected)
Mesh piece 3 OUTSIDE all meshes (corrected)
Mesh piece 4 OUTSIDE all meshes (corrected)
Mesh piece 5 OUTSIDE all meshes (corrected)
Mesh piece 6 OUTSIDE all meshes (corrected)
mesh pieces in subtact soma AFTER the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Signifiant mesh pieces of 50 size after poisson = 638
     Starting Calcification
-----Time for Running Calcification = 68.9192955493927
Before mesh subtraction number of skeleton edges = 5630
Inside mesh subtraction, len(main_mesh_bbox_restricted.faces) = 514598


...


Total Mesh subtraction time = 0.1165



No skeletons to stack so returning empty list
only one skeleton so no stacking needed
After cgal process the un-stitched skeleton has shape (18, 2, 3)
len_subgraphs AT BEGINNING of the loop


all graph is one component!
Total time for skeleton stitching = 0.030240297317504883
soma_connecting_skeleton[0].shape = (1, 2, 3)
total_soma_skeletons[0].shape = (19913, 2, 3)
mesh_soma_list = [<trimesh.Trimesh(vertices.shape=(2194, 3), faces.shape=(4191, 3))>]
mesh_soma_list_indexes = [3]
mesh_soma_list_centers = [array([314052.09630811, 874881.24380127, 803389.36868733])]



 Inside New skeletonization recursive calls



inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(689, 3), faces.shape=(1526, 3))>]
There were 1 pieces found after size threshold
Mesh piece 0 inside mesh 0 :( 
mesh pieces in subtact soma AFTER the filtering inside pieces = []
Total Time for soma mesh cancellation = 0.015
currently working on soma index 3
mesh_pieces after the soma subtraction = 0
No significant pieces after soma cancellation so just using the soma center as the skeleton
Total time for soma touching skeletons: 971.62478804588

all graph is one component!

Total time for skeleton stitching = 0.9419503211975098
Using Distance measure skeletal_distance


Total time for skeleton clean 188.30266690254211




Total time for whole skeletonization of neuron = 1161.9841530323029


In [52]:
sk.graph_skeleton_and_mesh(
    current_neuron.vertices,
    current_neuron.faces,
    edge_coordinates=multi_soma_skeleton,
    html_path=export_name+".html"
    
)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [53]:
sk.save_skeleton_cgal(multi_soma_skeleton,export_name + "_skeleton.cgal")

# Running Multiple tests of different multi-soma neurons

In [54]:
seg_ids = [78347281585498227,77504506459698055]
names = ["_another_5_soma_without_glia","_4_soma_little_glia"]

for export_name,segment_id in zip(names,seg_ids):


    current_neuron_path = "../test_neurons/" + f"{segment_id}{export_name}.off"

    current_neuron = trimesh.load_mesh(current_neuron_path)
    
    soma_mesh_list,run_time,total_soma_list_sdf = soma_utils.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)

    total_soma = sk.combine_meshes(soma_mesh_list)
    total_soma.show()
    
    multi_soma_skeleton = sk.skeletonize_neuron(main_mesh_total=current_neuron,
                        segment_id = segment_id,
                        soma_mesh_list = soma_mesh_list,
                       mesh_base_path="",
                       current_name="",
                       filter_end_node_length=5000,
                       sig_th_initial_split=15,

                        )
    sk.save_skeleton_cgal(multi_soma_skeleton,export_name + "_skeleton.cgal")

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 848
xvfb-run -n 848 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227.off -o /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/78347281585498227/decimation_meshlab_25605155.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(142227, 3), faces.shape=(297498, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(142227, 3), faces.shape=(297498, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decim

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(58229, 3), faces.shape=(116462, 3))>, <trimesh.Trimesh(vertices.shape=(48347, 3), faces.shape=(96694, 3))>, <trimesh.Trimesh(vertices.shape=(15694, 3), faces.shape=(31384, 3))>, <trimesh.Trimesh(vertices.shape=(12071, 3), faces.shape=(24146, 3))>, <trimesh.Trimesh(vertices.shape=(5823, 3), faces.shape=(11642, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(58229, 3), faces.shape=(116462, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 5215
xvfb-run -n 5215 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/78347281585498227/decimation_

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_581718.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_581718_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_414286.mls is being deleted....
Inside sphere validater: ratio_val = 2.481051385999243
----- working on mesh after poisson #1: <trimesh.Trimesh(vertices.shape=(48347, 3), faces.shape=(96694, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8321
xvfb-run -n 8321 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/78347281585498227/decimation_meshlab_25811013.mls
done exporting deci

...


Total Mesh subtraction time = 0.3923



After cgal process the un-stitched skeleton has shape (141, 2, 3)
len_subgraphs AT BEGINNING of the loop


all graph is one component!
Total time for skeleton stitching = 0.054978132247924805
Time for non-soma skeletons = 6.423409700393677
only one skeleton so no stacking needed
only one skeleton so no stacking needed

len_subgraphs AT BEGINNING of the loop


all graph is one component!

Total time for skeleton stitching = 1.6420915126800537
Using Distance measure skeletal_distance


Total time for skeleton clean 529.1308410167694




Total time for whole skeletonization of neuron = 1490.841801404953


In [55]:
seg_ids = [78347281585498227,77504506459698055]
names = ["_another_5_soma_without_glia","_4_soma_little_glia"]
for export_name,segment_id in zip(names,seg_ids):
    multi_soma_skeleton = sk.read_skeleton_edges_coordinates(export_name + "_skeleton.cgal")
    
    sk.graph_skeleton_and_mesh(
    current_neuron.vertices,
    current_neuron.faces,
    edge_coordinates=multi_soma_skeleton,
    html_path=export_name+".html"
    
    )

only one skeleton so no stacking needed


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


only one skeleton so no stacking needed


# running single skeletonization again

In [56]:
# seg_ids = [78347281585498227,77504506459698055]
# names = ["_another_5_soma_without_glia","_4_soma_little_glia"]

# for export_name,segment_id in zip(names,seg_ids):

export_name = "_another_5_soma_without_glia"
segment_id = 78347281585498227

current_neuron_path = "../test_neurons/" + f"{segment_id}{export_name}.off"

current_neuron = trimesh.load_mesh(current_neuron_path)

soma_mesh_list,run_time,total_soma_list_sdf = soma_utils.extract_soma_center(segment_id,
                                         current_neuron.vertices,
                                         current_neuron.faces)

total_soma = sk.combine_meshes(soma_mesh_list)
total_soma.show()

multi_soma_skeleton = sk.skeletonize_neuron(main_mesh_total=current_neuron,
                    segment_id = segment_id,
                    soma_mesh_list = soma_mesh_list,
                   mesh_base_path="",
                   current_name="",
                   filter_end_node_length=5000,
                   sig_th_initial_split=15,

                    )

sk.graph_skeleton_and_mesh(
current_neuron.vertices,
current_neuron.faces,
edge_coordinates=multi_soma_skeleton,
html_path=export_name+".html"

)
sk.save_skeleton_cgal(multi_soma_skeleton,export_name + "_skeleton.cgal")

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 8945
xvfb-run -n 8945 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227.off -o /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/78347281585498227/decimation_meshlab_25808098.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(142227, 3), faces.shape=(297498, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(142227, 3), faces.shape=(297498, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_dec

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(58229, 3), faces.shape=(116462, 3))>, <trimesh.Trimesh(vertices.shape=(48347, 3), faces.shape=(96694, 3))>, <trimesh.Trimesh(vertices.shape=(15694, 3), faces.shape=(31384, 3))>, <trimesh.Trimesh(vertices.shape=(12071, 3), faces.shape=(24146, 3))>, <trimesh.Trimesh(vertices.shape=(5823, 3), faces.shape=(11642, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(58229, 3), faces.shape=(116462, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 6854
xvfb-run -n 6854 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/78347281585498227/decimation_

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_951718.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_951718_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_465926.mls is being deleted....
Inside sphere validater: ratio_val = 2.481051385999243
----- working on mesh after poisson #1: <trimesh.Trimesh(vertices.shape=(48347, 3), faces.shape=(96694, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 9801
xvfb-run -n 9801 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/78347281585498227/neuron_78347281585498227_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/78347281585498227/decimation_meshlab_25593406.mls
done exporting deci

...


Total Mesh subtraction time = 0.4469



After cgal process the un-stitched skeleton has shape (129, 2, 3)
len_subgraphs AT BEGINNING of the loop


all graph is one component!
Total time for skeleton stitching = 0.060328006744384766
Time for non-soma skeletons = 6.304066181182861
only one skeleton so no stacking needed
only one skeleton so no stacking needed

len_subgraphs AT BEGINNING of the loop


all graph is one component!

Total time for skeleton stitching = 1.2256906032562256
Using Distance measure skeletal_distance


Total time for skeleton clean 395.46458435058594




Total time for whole skeletonization of neuron = 1422.0154538154602


In [57]:
# seg_ids = [78347281585498227,77504506459698055]
# names = ["_another_5_soma_without_glia","_4_soma_little_glia"]

# for export_name,segment_id in zip(names,seg_ids):

export_name = "_4_soma_little_glia"
segment_id = 77504506459698055

current_neuron_path = "../test_neurons/" + f"{segment_id}{export_name}.off"

current_neuron = trimesh.load_mesh(current_neuron_path)

soma_mesh_list,run_time,total_soma_list_sdf = soma_utils.extract_soma_center(segment_id,
                                         current_neuron.vertices,
                                         current_neuron.faces)

total_soma = sk.combine_meshes(soma_mesh_list)
total_soma.show()

multi_soma_skeleton = sk.skeletonize_neuron(main_mesh_total=current_neuron,
                    segment_id = segment_id,
                    soma_mesh_list = soma_mesh_list,
                   mesh_base_path="",
                   current_name="",
                   filter_end_node_length=5000,
                   sig_th_initial_split=15,

                    )

sk.graph_skeleton_and_mesh(
current_neuron.vertices,
current_neuron.faces,
edge_coordinates=multi_soma_skeleton,
html_path=export_name+".html"

)
sk.save_skeleton_cgal(multi_soma_skeleton,export_name + "_skeleton.cgal")

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 2651
xvfb-run -n 2651 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/77504506459698055/neuron_77504506459698055.off -o /notebooks/Platinum_Skeletonization_vp3/77504506459698055/neuron_77504506459698055_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/77504506459698055/decimation_meshlab_25431171.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(173271, 3), faces.shape=(366642, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(173271, 3), faces.shape=(366642, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Skeletonization_vp3/77504506459698055/neuron_77504506459698055_dec

face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_normals all zero, ignoring!
face_norma

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(78865, 3), faces.shape=(157810, 3))>, <trimesh.Trimesh(vertices.shape=(35492, 3), faces.shape=(70984, 3))>, <trimesh.Trimesh(vertices.shape=(19290, 3), faces.shape=(38580, 3))>, <trimesh.Trimesh(vertices.shape=(13193, 3), faces.shape=(26382, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(78865, 3), faces.shape=(157810, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 105
xvfb-run -n 105 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/77504506459698055/neuron_77504506459698055_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Skeletonization_vp3/77504506459698055/neuron_77504506459698055_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Skeletonization_vp3/77504506459698055/decimation_meshlab_25693929.mls
done exporting decimated mesh: neuron_775045064596

face_normals all zero, ignoring!


removed temporary input file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_517970.off
removed temporary output file: /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_517970_poisson.off
/notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_451065.mls is being deleted....
Inside sphere validater: ratio_val = 3.6147035384231514
Using Poisson Surface Reconstruction to make mesh watertight
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 3785
xvfb-run -n 3785 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_607849.off -o /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/neuron_607849_poisson.off -s /notebooks/Platinum_Skeletonization_vp3/Poisson_temp/poisson_947204.mls


...


Total Mesh subtraction time = 0.4033



After cgal process the un-stitched skeleton has shape (147, 2, 3)
len_subgraphs AT BEGINNING of the loop


all graph is one component!
Total time for skeleton stitching = 0.04565310478210449
Time for non-soma skeletons = 6.502452373504639
only one skeleton so no stacking needed
only one skeleton so no stacking needed

len_subgraphs AT BEGINNING of the loop


all graph is one component!

Total time for skeleton stitching = 1.571455478668213
Using Distance measure skeletal_distance


Total time for skeleton clean 528.4793062210083




Total time for whole skeletonization of neuron = 1492.6700570583344
